In [1]:
import pandas as pd

train_data_s3path = "s3://sagemaker-us-east-1-980831117329/data/boston/train/data"
pd.read_csv(train_data_s3path,
            names=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS",
                   "RAD", "TAX", "PTRATIO", "B", "LSTAT", "price"])

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price
0,28.7,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
1,17.0,1.41385,0.0,19.58,1.0,0.871,6.129,96.0,1.7494,5.0,403.0,14.7,321.02,15.12
2,11.9,20.71620,0.0,18.10,0.0,0.659,4.138,100.0,1.1781,24.0,666.0,20.2,370.22,23.34
3,43.5,0.54050,20.0,3.97,0.0,0.575,7.470,52.6,2.8720,5.0,264.0,13.0,390.30,3.16
4,14.5,0.98843,0.0,8.14,0.0,0.538,5.813,100.0,4.0952,4.0,307.0,21.0,394.54,19.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,13.4,6.71772,0.0,18.10,0.0,0.713,6.749,92.6,2.3236,24.0,666.0,20.2,0.32,17.44
280,13.3,0.24980,0.0,21.89,0.0,0.624,5.857,98.2,1.6686,4.0,437.0,21.2,392.04,21.32
281,19.4,0.26363,0.0,8.56,0.0,0.520,6.229,91.2,2.5451,5.0,384.0,20.9,391.23,15.55
282,12.5,5.87205,0.0,18.10,0.0,0.693,6.405,96.0,1.6768,24.0,666.0,20.2,396.90,19.37


In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = get_execution_role()

container = sagemaker.image_uris.retrieve("linear-learner", boto3.Session().region_name)

In [3]:
from time import gmtime, strftime

job_name = f"boston-house-price-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
linear = sagemaker.estimator.Estimator(container,
                                       role,
                                       instance_count=1,
                                       instance_type='ml.c4.xlarge',
                                       sagemaker_session=sagemaker.Session())

linear.set_hyperparameters(feature_dim=13,
                           mini_batch_size=100,
                           predictor_type='regressor')

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_data_s3path, content_type='text/csv')
linear.fit(
    {"train": s3_input_train},
    job_name=job_name
)

2022-07-26 04:09:13 Starting - Starting the training job...
2022-07-26 04:09:40 Starting - Preparing the instances for trainingProfilerReport-1658808553: InProgress
.........
2022-07-26 04:11:13 Downloading - Downloading input data......
2022-07-26 04:12:13 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/26/2022 04:12:12 INFO 140426084742976] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss': 'auto', 'margin': '1.0', '

In [4]:
model_name = f"{job_name}-model"
print(model_name)

region = boto3.Session().region_name
client = boto3.client("sagemaker", region_name=region)
info = client.describe_training_job(TrainingJobName=job_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]
print(model_data)

primary_container = {"Image": container, "ModelDataUrl": model_data}

create_model_response = client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

boston-house-price-2022-07-26-04-09-13-model
s3://sagemaker-us-east-1-980831117329/boston-house-price-2022-07-26-04-09-13/output/model.tar.gz
arn:aws:sagemaker:us-east-1:980831117329:model/boston-house-price-2022-07-26-04-09-13-model


In [5]:
endpoint_config_name = f"boston-house-price-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
client = boto3.client("sagemaker", region_name=region)

print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print(f"Endpoint Config Arn: {create_endpoint_config_response['EndpointConfigArn']}")

boston-house-price-2022-07-26-04-13-00
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:980831117329:endpoint-config/boston-house-price-2022-07-26-04-13-00


In [6]:
import time 
endpoint_name = f"boston-house-price-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
print(endpoint_name)

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
while status == "Creating":
    print(f"Status: {status}")
    time.sleep(60)
    resp = client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]

boston-house-price-2022-07-26-04-13-00
arn:aws:sagemaker:us-east-1:980831117329:endpoint/boston-house-price-2022-07-26-04-13-00
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
